In [ ]:
import scipy.io as sio
import scipy.sparse as spp
import numpy as np
import scipy as sp
from sklearn.preprocessing import normalize
import json

def extract_rows(top_k, sparse_matrix):
    business_review_count=sparse_matrix.getnnz(axis=1)
    business_count=business_review_count.shape[0]
    top_k_index = np.argsort(business_review_count)[business_count-1: business_count -1 -top_k: -1]
    # top_k_index = np.random.choice(business_count, top_k, replace=False)
    matrix=spp.vstack([sparse_matrix.getrow(i) for i in top_k_index])
    return matrix

def extract_cols(top_k, sparse_matrix):
    user_review_count=sparse_matrix.getnnz(axis=0)
    user_count=user_review_count.shape[0]

    top_k_index=np.argsort(user_review_count)[user_count-1: user_count-1-top_k:-1]
    # top_k_index=np.random.choice(user_count, top_k, replace=False)
    matrix=spp.hstack([sparse_matrix.getcol(i) for i in top_k_index])
    return matrix

def load_sparse_matrix(file_name):
    data_list = []
    row_indics_list = []
    col_indics_list = []

    user_dict = {}
    business_dict = {}

    rf = open(file_name)

    l = rf.readline()
    count = 0
    for line in rf:
        dicts = json.loads(line)
        row_index = 0
        col_index = 0
        user_id = dicts["user_id"]
        business_id = dicts["business_id"]
        rating = float(dicts["stars"])

        if not user_id in user_dict: #.has_key(user_id):
            user_dict[user_id] = len(user_dict)
        row_index = user_dict[user_id]

        if not business_id in business_dict: #.has_key(business_id):
            business_dict[business_id] = len(business_dict)
        col_index = business_dict[business_id]

        #data_list.append(float(rating))
        if rating > 4.0:
            data_list.append(1)
        elif rating < 2.0:
            data_list.append(-1)
            count +=1
        else: 
            continue 
            
        row_indics_list.append(row_index)
        col_indics_list.append(col_index)

    data = np.array(data_list)
    rows = np.array(row_indics_list)
    cols = np.array(col_indics_list)
    print(len(user_dict), len(business_dict))
    s_m = spp.csr_matrix((data, (rows, cols)))
    print("negative samples:", count)
    return s_m


def get_reduced_concrete_matrix(user_num, business_num,filename):
    s_m = load_sparse_matrix(filename)
    print(s_m.shape)
    row_reduced_matrix=extract_rows(user_num*3, s_m)
    reduced_matrix=extract_cols(business_num, row_reduced_matrix)
    reduced_matrix = extract_rows(user_num, reduced_matrix)
    return reduced_matrix.toarray()


In [ ]:
filename = '../Yelp/review.json'
m = get_reduced_concrete_matrix(2000, 10000, filename)
print(m.shape)



In [ ]:
new_m = []
for i in range(len(m)):
    for j in range(len(m[0])):
        if m[i][j] > 0:
            new_m.append((i,j,1))
            
        if m[i][j] < 0:
            new_m.append((i,j,-1))

np.save("../Yelp/yelp_2000users_10000items_entry.npy", np.array(new_m))

In [ ]:
np.save("../Yelp/yelp_2000users_10000items.npy", m)
C = m.T
np.save("../Yelp/yelp_10000items_2000users.npy", C)

In [ ]:
def ExtractFeatures(num, d, filename):
    X = np.load(filename)

    A1 = X[:num, :]
    u, s, vt = np.linalg.svd(A1)

    u = u[:, :d-1]
    u = normalize(u, axis = 1, norm = 'l2')
    # print(np.linalg.norm(u[0,:]))

    U = np.concatenate((u, np.ones((num, 1))), axis = 1) / np.sqrt(2)
    return U

U = ExtractFeatures(2000, d=10, filename="../Yelp/yelp_2000users_10000items.npy")
np.save("../Yelp/yelp_2000users_10000items_features.npy", U)

I = ExtractFeatures(10000, d=10, filename="../Yelp/yelp_10000items_2000users.npy")
np.save("../Yelp/yelp_10000items_2000users_features.npy", I)

